In [21]:
import numpy as np
import pandas as pd
import os
import re
from googletrans import Translator
from IPython.display import display, Image
from functools import reduce
import json



In [22]:
translator = Translator()


In [23]:
path_to_dir ='isa-data/xlsx'
list_df_final = []
i = -1
# Dictionary used to associate a dummy sciper to students with missing Sciper number
name_sciper= {}

# Dictionary used to normalize sections names
codes = {'AR':'Architecture','CGC':'Chemical Engineering and Biotechnology',
         'GC':'Civil Engineering','EL':'Electrical and Electronics Engineering',
         'GM':'Mechanical Engineering','IN':'Computer Science','IF':'Financial engineering',
         'MA':'Mathematics','MT':'Microengineering',
         'MX':'Materials Science and Engineering',
         'SC': 'Communication Systems','SIE':'Environmental Sciences and Engineering',
         'SV':'Life Sciences and Technologies','MTE':'Management','PH':'Physics'}

def getSciper(key):
    ''' Function that assign a dummy sciper to a student without sciper number in our database'''
    global i,name_sciper
    try:
        return name_sciper[key]
    except KeyError:
        name_sciper[key] = i
        ret = i
        i-=1
        return ret
def clean_type(type_):
    ''' Function used to parse the headers and clean some fields'''
    global i
    
    # Check if header exists
    if(type_['Level'] is None):
        return type_
    # Split header 
    temp= type_['Level'].split(',')
    
    # Collect the section name from the header
    type_['section'] = temp[0]
    level = temp[2] if (temp[0] != 'Management') else temp[3]
    type_['Level']= re.sub(r"\xa0(.)*", r"",level).strip()
    
    '''Clean and normalize the nationality field in order to keep only one nationality in the case where a student has
    multiples ones '''
    
    if(type(type_['Nationalité']) == str):
        if(('Suisse' in type_['Nationalité']) or ('Swiss' in  type_['Nationalité'])):
            type_['Nationalité']='Swiss'
        else:
            temp = re.split(r"(,+|;+)", type_['Nationalité'])
            if(len(temp) > 1):
                type_['Nationalité']=temp[0].strip()
                
    # Determine if the current entry contains infos about an exchange student 
    exchange = 'Exchange' in type_['section'] or 'Echange' in type_['section']
    type_['Exchange_student'] = exchange
    if(exchange):
        type_['section']=codes[re.sub(r"Exchange|Echange", r"",type_['section']).strip()]
        
    type_['section']= re.sub(r"-(.)*", r"",type_['section']).strip()
    
    # Get dummy sciper if the latter is not defined
    if(type(type_['No Sciper']) != int ):
        type_['No Sciper'] = getSciper(type_['Nom Prénom'])
        
    # Determine the type of student
    if type_['Exchange_student']:
        type_['Type']='Exchange'
    elif type_['section'] == 'CMS':
        type_['Type']='CMS'
    elif type_['section'] == 'Passerelle HES':
        type_['Type']= 'Passerelle HES'
    elif 'Master' in type_['Level']:
        type_['Type']= 'Master'
    elif ('Bachelor' in type_['Level']) or ('training course' in type_['Level']):
        type_['Type']= 'Bachelor'
    
    return type_

def process_df(df,year):
    ''' Function that processes individually every excel file'''
    
    # Get headers
    df.columns = df.loc[1,:].tolist()
    df = df[df['Civilité']!='Civilité'].reset_index(drop=True)
    df['Level'] = None
    types = df[ (df['Civilité']!='Miss') & (df['Civilité']!='Mister')]['Civilité']
    types_index = types.index.tolist()
    types = types.tolist()
    
    # Associate every entry to its corresponding header
    for i in range(len(types_index)):
        index_2 = 0
        if(i != len(types_index)-1):
            index_2= types_index[i+1]
        else:
            index_2 = len(df)-1

        df.loc[types_index[i]:index_2,'Level']= types[i]

    # Drop headers
    df = df.loc[(df['Civilité']=='Miss')|(df['Civilité']=='Mister')].reset_index(drop = True)
    df['Year']= year
    df['section'] = None
    df['Exchange_student']= None
    df['Type']= None
    
    # Call clean_type function which will parse the header to extract the Year,section,Type
    #(Bachelor,Master,Exchange, CMS ,passerelle HES ) of every entry
    df = df.apply(clean_type,axis=1)
    return df

In [24]:
# Loading and preprocessing all excel files
for file in os.listdir(path_to_dir):
    df= pd.read_excel(os.path.join(path_to_dir,file))
    year_df = process_df(df,file.split('.')[0])
    list_df_final.append(year_df)

In [25]:

def unique_entries(df):
    ''' Function used to list a student only once in a certain year dataframe'''
    if((df['Nom Prénom'] != df['Nom Prénom'].iloc[0]).any()):
        display(df)
    ret = df[~(df['Type'].isnull())]
    if(len(ret)>0):
        ret = ret.head(1)
    else:
        ret= df.head(1)
    return ret

# group the entries in every excel file by sciper number and use the unique_entries function to reduce them
list_df_final = list(map(lambda x : (x.groupby(by='No Sciper')).apply(unique_entries),list_df_final))
reduce(lambda x,y:x+y ,list(map(lambda x:len(x),list_df_final)))




69262

In [26]:
list_df_final = list(map(lambda x : x.reset_index(drop = True),list_df_final))



In [27]:
# Concatenate all loaded dataframes into a unique one
final_df = pd.concat(list_df_final,ignore_index=True)

# Normalizing sections names and nationalities + other fields 
final_df.loc[final_df.section == 'Management of Technology','section'] = 'Management'
final_df.loc[final_df.section == 'Systèmes de communication','section'] = 'Communication Systems'
final_df.replace(u'\xa0',u' ', regex=True, inplace=True)
final_df['Nationalité'].replace(u'suisse',u'Swiss', regex=True, inplace=True)
final_df['Nationalité'].replace(u';',u',', regex=True, inplace=True)

# try to find sciper numbers for entries which does not have one using informations from the others dataframes.
# If a valid sciper number is not found we associate the same dummy sciper for entries with the same name attribute
bad_entries = final_df[final_df['No Sciper']< 0]
bad_entries_names = bad_entries['Nom Prénom'].unique()
for name in bad_entries_names:
    sciper = final_df[(final_df['No Sciper']>0) & (final_df['Nom Prénom'] == name)]
    if(len(sciper)>0):
        final_df.loc[(final_df['No Sciper']<0) & (final_df['Nom Prénom'] == name),'No Sciper']= (sciper['No Sciper'].reset_index(drop=True))[0]

In [31]:
# Dictionary used to map every nationality with its country code , this step is necessary to display nationality infos in Leaflet maps
# At this step the dictionary only associates to every nationality in our df it correspendant name in english
mapping_code_nat = dict(list(map(lambda x: (x,translator.translate(x,dest = 'en').text.title()) if type(x) == str else (x,x),final_df['Nationalité'].unique().tolist())))


In [33]:
# Handle manually some nationalities names which were badly translated

mapping_code_nat['kosovar'] = 'Kosovar'
mapping_code_nat['centrafricaine']= 'Central African'
mapping_code_nat['Philippine']= 'Filipino'
mapping_code_nat['néo-zélandaise'] = 'New Zealander'
mapping_code_nat['liechtensteinoise']='Liechtensteiner'
mapping_code_nat['Qatar']= 'Qatari'
mapping_code_nat['lettonne']= 'Latvian'
mapping_code_nat['Bangladesh']= 'Bangladeshi'

# uniformize nationalities
def f(row):
    global mapping_code_nat,csv_code
    if(row['Nationalité'] is not None) and (type(row['Nationalité'])== str):
        row['Nationalité'] = mapping_code_nat[row['Nationalité']]
    return row
final_df_copy= final_df.apply(f,axis=1)

In [38]:

# Load a csv files found on internet which associates nationalities to countries codes
csv_codes = pd.read_csv('isa-data/Countries-List.csv',encoding='utf-16')
csv_codes = csv_codes.set_index('Demonym_1').to_dict('index')


In [90]:
# Save our dictionary that maps countries code to citizenships
nat_code = {v['ISO_Code']: k for k, v in csv_codes.items()}

nat_code['YF'] = 'Serbian'
nat_code['ZR'] = 'Congolese'
nat_code['GN'] = 'Guinean'
nat_code['NA'] = 'Namibian'
nat_code['SD'] = 'Sudanese'
nat_code['GZ'] = 'Israeli'
nat_code['TP'] = 'East Timorian'
nat_code['RY'] = 'Yemeni'

js = json.dumps(nat_code)
fp = open('code_nat.json', 'a')
fp.write(js)
fp.close()

# Save our dictionary that maps citizenships to countries names
nat_country ={}
config = json.loads(open('isa-data/custom.geo.json').read())

    
for i in  config['features']:
    code = None
    if (i['properties']['wb_a2'] != '-99'):
        code = i['properties']['wb_a2']
    else:
        code = i['properties']['postal']
    if(code not in nat_code):
        print(i['properties']['sovereignt'],code)
    else:
        nat_country[nat_code[code]] = i['properties']['sovereignt']
    if(code == '-99'):
        print(i)
        


js = json.dumps(nat_country)
fp = open('nat_country.json', 'a')
fp.write(js)
fp.close()

In [50]:
# Save the processed dataframe and keep only columns that will be useful in our data visualization
final_df_copy[['Civilité','Year','Type','section','Nationalité']].to_csv('cleaned_isa_data.csv')